In [30]:
import numpy as np
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.applications.resnet50 import ResNet50 ## appliction is in tensorflow not in keras

In [31]:
data=pd.read_csv("tagets_and_imagename.csv")
data.head

<bound method NDFrame.head of        age  gender                              image_path
0      100       0  100_0_0_20170112213500903.jpg.chip.jpg
1      100       0  100_0_0_20170112215240346.jpg.chip.jpg
2      100       1  100_1_0_20170110183726390.jpg.chip.jpg
3      100       1  100_1_0_20170112213001988.jpg.chip.jpg
4      100       1  100_1_0_20170112213303693.jpg.chip.jpg
...    ...     ...                                     ...
23704    9       1    9_1_3_20161220222856346.jpg.chip.jpg
23705    9       1    9_1_3_20170104222949455.jpg.chip.jpg
23706    9       1    9_1_4_20170103200637399.jpg.chip.jpg
23707    9       1    9_1_4_20170103200814791.jpg.chip.jpg
23708    9       1    9_1_4_20170103213057382.jpg.chip.jpg

[23709 rows x 3 columns]>

In [32]:
## randomizing the data using sample() function
train_data=data.sample(frac=1,random_state=0)[:20000]
test_data=data.sample(frac=1,random_state=0)[20000:]

In [33]:
from tensorflow.keras.preprocessing.image import img_to_array, load_img

## genrator for image data(train)
dir_path=".\\archive\\utkcropped\\utkcropped\\"
batch_size=32
def genrator_train_data():
    for i in range(0,train_data.shape[0],batch_size):
        image_list=[]
        target_age=[]
        target_gender=[]
        for j in range(i,min(i+batch_size,train_data.shape[0])):
            
            image_Path=dir_path+train_data["image_path"][j]
            
            image = load_img(image_Path, target_size=(200, 200))
            img=img_to_array(image)
            img=img/255.
            
            target_age.append(train_data["age"][j])
            target_gender.append(train_data["gender"][j])
            image_list.append(img)
            
        yield((np.array(image_list)),np.array(target_age),np.array(target_gender))
    

In [34]:
## genrator for image data(test)
dir_path=".\\archive\\utkcropped\\utkcropped\\"
batch_size=32
def genrator_test_data():
    for i in range(0,test_data.shape[0],batch_size):
        image_list=[]
        target_age=[]
        target_gender=[]
        for j in range(i,min(i+batch_size,test_data.shape[0])):
            
            image_Path=dir_path+test_data["image_path"][j]
            
            image = load_img(image_Path, target_size=(200, 200))
            img=img_to_array(image)
            img=img/255.
            
            target_age.append(test_data["age"][j])
            target_gender.append(test_data["gender"][j])
            image_list.append(img)
            
        yield((np.array(image_list)),np.array(target_age),np.array(target_gender))

In [35]:
train_data_genrator=genrator_train_data()
# test_data_genrator=genrator_test_data()

In [36]:
# next(train_data_genrator)

KeyError: 10

In [7]:
train_data_genrator

<generator object genrator_train_data at 0x000001EE3E1B52A0>

In [8]:
# from keras.preprocessing.image import ImageDataGenerator

In [9]:
# train_datagen = ImageDataGenerator(rescale=1./255,
#                                    rotation_range=30,
#                                    width_shift_range=0.2,
#                                    height_shift_range=0.2,
#                                    shear_range=0.2,
#                                    zoom_range=0.2,
#                                    horizontal_flip=True)

# test_datagen = ImageDataGenerator(rescale=1./255)

In [10]:
# train_generator = train_datagen.flow_from_dataframe(train_data,
#                                                     directory=".\\archive\\utkcropped\\utkcropped",
#                                                     x_col='image_path',
#                                                     y_col=['age','gender'],
#                                                     target_size=(200,200),
#                                                     class_mode='multi_output')

# test_generator = test_datagen.flow_from_dataframe(test_data,
#                                                     directory=".\\archive\\utkcropped\\utkcropped",
#                                                     x_col='image_path',
#                                                     y_col=['age','gender'],
#                                                     target_size=(200,200),
#                                                   class_mode='multi_output')

In [11]:
# train_generator

In [12]:

from keras.layers import *
from keras.models import Model

In [13]:
resnet = ResNet50(include_top=False, input_shape=(200,200,3))

In [14]:
resnet = ResNet50(include_top=False, input_shape=(200,200,3))

resnet.trainable=False

output = resnet.layers[-1].output

flatten = Flatten()(output)

dense1 = Dense(512, activation='relu')(flatten)
dense2 = Dense(512,activation='relu')(flatten)

dense3 = Dense(512,activation='relu')(dense1)
dense4 = Dense(512,activation='relu')(dense2)

output1 = Dense(1,activation='linear',name='age')(dense3)
output2 = Dense(1,activation='sigmoid',name='gender')(dense4)

In [15]:
model = Model(inputs=resnet.input,outputs=[output1,output2])

In [16]:
model.compile(optimizer='adam', loss={'age': 'mae', 'gender': 'binary_crossentropy'}, metrics={'age': 'mae', 'gender': 'accuracy'},loss_weights={'age':1,'gender':99})

In [17]:
tf.config.list_physical_devices("GPU")

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [18]:
model.fit(train_data_genrator, epochs=10)

KeyError: 10